In [ ]:
import requests
import re
from time import sleep
from bs4 import BeautifulSoup

import pandas as pd
import tqdm

In [ ]:
def scraping_web_page(url):
    sleep(5)
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

In [ ]:
# B'zの歌詞は2ページ
urls = [
    'https://www.uta-net.com/artist/134/',
    'https://www.uta-net.com/artist/134/0/2/'
]
artist_dfs = []
for url in urls:
    #曲一覧ページをスクレイピングする
    soup = scraping_web_page(url)
    #htmlをパースして曲名、各曲URL、アーティスト名、作詞、作曲者名を取得する
    contents = []
    contents.append(soup.find_all(href=re.compile('/song/\d+/$')))
    contents.append(soup.find_all(href=re.compile('/song/\d+/$')))
    contents.append(soup.find_all(class_=re.compile('td2')))
    contents.append(soup.find_all(class_=re.compile('td3')))
    contents.append(soup.find_all(class_=re.compile('td4')))
    infomations = []
    for i, content in enumerate(contents):
        tmp_list = []
        for element in content:
            if i == 0:
                tmp_list.append(element.get('href'))
            else:
                tmp_list.append(element.string)
        infomations.append(tmp_list)
    #DataFrameにする
    artist_df = pd.DataFrame({
        'URL': infomations[0],
        'SongName': infomations[1],
        'Artist': infomations[2],
        'Lyricist': infomations[3],
        'Composer': infomations[4]
    })
    #URLにホストネームを付加
    artist_df['URL'] = artist_df['URL'].apply(lambda x: 'https://www.uta-net.com' + x)
    artist_dfs.append(artist_df)
df_all = pd.concat(artist_dfs)

In [ ]:
df_all

,URL,SongName,Artist,Lyricist,Composer
0,https://www.uta-net.com/song/54892/,I WANNA DANCE Wicked Beat Style,B'z,Kohshi Inaba,Takahiro Matsumoto
1,https://www.uta-net.com/song/7938/,あいかわらずなボクら,B'z,稲葉浩志,松本孝弘
2,https://www.uta-net.com/song/17979/,愛と憎しみのハジマリ,B'z,KOSHI INABA,TAK MATSUMOTO
3,https://www.uta-net.com/song/25415/,愛のバクダン,B'z,稲葉浩志,松本孝弘
4,https://www.uta-net.com/song/26049/,愛のprisoner,B'z,KOSHI INABA,TAK MATSUMOTO
...,...,...,...,...,...
142,https://www.uta-net.com/song/59207/,わるいゆめ,B'z,KOSHI INABA,TAK MATSUMOTO
143,https://www.uta-net.com/song/12871/,ONE,B'z,稲葉浩志,松本孝弘
144,https://www.uta-net.com/song/59204/,ONE ON ONE,B'z,KOSHI INABA,TAK MATSUMOTO
145,https://www.uta-net.com/song/6977/,Wonderful Opportunity,B'z,稲葉浩志,松本孝弘


In [ ]:
# 各曲のページをスクレイピングする
lyrics = []
for i, url in tqdm.notebook.tqdm(df_all['URL'].iteritems(), total=len(df_all)):
  content = scraping_web_page(url)
  lyrics.append(content.find(id='kashi_area').text)

In [ ]:
df_all['lyrics'] = lyrics
df_all

,URL,SongName,Artist,Lyricist,Composer,lyrics
0,https://www.uta-net.com/song/54892/,I WANNA DANCE Wicked Beat Style,B'z,Kohshi Inaba,Takahiro Matsumoto,Crush! Crush! Taxi on a busy neon streetGive m...
1,https://www.uta-net.com/song/7938/,あいかわらずなボクら,B'z,稲葉浩志,松本孝弘,今まで好きなこともしたし たまに我慢もしてきた「あいつはダメだ」 なんてキミ勝手に決めないで...
2,https://www.uta-net.com/song/17979/,愛と憎しみのハジマリ,B'z,KOSHI INABA,TAK MATSUMOTO,誰もが愛する者 守ろうとしてる頃 憎しみがスキをうかがう大破綻の雰囲気が世界中を取り巻いて朝...
3,https://www.uta-net.com/song/25415/,愛のバクダン,B'z,稲葉浩志,松本孝弘,今日も一日 不完全な世界きみが泣かなくても 誰か泣く人がこしらえた 悲しみのからくりなんとか...
4,https://www.uta-net.com/song/26049/,愛のprisoner,B'z,KOSHI INABA,TAK MATSUMOTO,"I hate you, hate you こらえきれずに夜の闇に吠える 涙流さないあなたのど..."
...,...,...,...,...,...,...
142,https://www.uta-net.com/song/59207/,わるいゆめ,B'z,KOSHI INABA,TAK MATSUMOTO,12時すぎてひとりで ムリヤリ見る話題作場面がだらだら流れて これ以上飲みこめませんおまえは...
143,https://www.uta-net.com/song/12871/,ONE,B'z,稲葉浩志,松本孝弘,静かすぎる夜だ 耳がうずくほど僕も君も誰も 眠ってなんかいないのに言葉がまだ 大切なこと伝え...
144,https://www.uta-net.com/song/59204/,ONE ON ONE,B'z,KOSHI INABA,TAK MATSUMOTO,Tシャツを脱いだって かまわないよ 暑ければそのポケットも空にして金網ごしに誰か 見てるけど...
145,https://www.uta-net.com/song/6977/,Wonderful Opportunity,B'z,稲葉浩志,松本孝弘,手も足も出ないような 悩みに縛られてひとりきりむりやり酔っぱらって アルバムを抱いて寝た目が...


In [ ]:
df_all.to_csv('/content/drive/MyDrive/other/generate_bz_lyrics/df_all.csv', index=False)